In [1]:
import pdfplumber

In [2]:
# creating a pdf file object 
# pdfFileObj = open('./data/am_one.pdf', 'rb'
# text = extract_text('./data/am_one.pdf')

In [5]:
pdf = pdfplumber.open("./data/am_one.pdf")
first_page = pdf.pages[26]
print(first_page.chars[0])

{'fontname': 'AdvPS6F00', 'adv': Decimal('0.614'), 'upright': True, 'x0': Decimal('137.820'), 'y0': Decimal('667.835'), 'x1': Decimal('143.937'), 'y1': Decimal('677.797'), 'width': Decimal('6.117'), 'height': Decimal('9.963'), 'size': Decimal('9.963'), 'object_type': 'char', 'page_number': 27, 'stroking_color': None, 'non_stroking_color': (0, 0, 0), 'text': 'T', 'top': Decimal('99.203'), 'bottom': Decimal('109.165'), 'doctop': Decimal('20499.370')}


In [105]:
start_page = 91
end_page = 420
first_page = pdf.pages[133]
text = first_page.extract_text(x_tolerance=1, y_tolerance=3)
# print(text)
# text.split('REFERENCES')
# text.replace('\n',' ').lower()
len([foo[2] for foo in text.split('\n') if foo[2] == '.']) > 10

True

In [113]:
# generating text

def is_references_page(page_text):
    lines = page_text.split('\n')
    counter = 0
    for line in lines:
        if len(line) > 30:
            if line[2] == '.' or line[3] == '.':
                counter = counter + 1
    if counter > 3:
        return True
    return False


all_pages_text = ''
counter_w = 0
for page_number in range(91, 420):
    first_page = pdf.pages[page_number]
    page_text = first_page.extract_text(x_tolerance=1, y_tolerance=3)
    if page_text:
        page_text = page_text.split('REFERENCES')[0]
        references_page = is_references_page(page_text)
        if not references_page:
            # counter_w = counter_w + 1
            # print("============")
            # print(page_text)
            page_text = page_text.replace('\n',' ').lower()
            all_pages_text = all_pages_text + page_text
    
# print(all_pages_text)
len(all_pages_text)
# counter_w

770639

In [33]:
# entites generator
# importing modules
import spacy
import scispacy
# import pandas as pd

from spacy.pipeline import EntityRuler


nlp = spacy.load("en_core_sci_lg")
# nlp = spacy.load("en_core_web_sm")

properties_ruler = EntityRuler(nlp).from_disk("./data/am.jsonl")
nlp.add_pipe(properties_ruler)

In [58]:
# entitties
doc = nlp(all_pages_text.lower())
entities = []
for ent in doc.ents:
    # print("------------")
    # if ent.label_ in ["AMPROP", "AMING"]:
    # print(ent.label_)
    # print(ent.text)
    entities.append([ent.text, ent.label_])

# entities = list(set(entities))
#len(entities)
# entities
# all_pages_text[0:10000]

In [59]:
import pandas as pd

entities_df = pd.DataFrame(entities)

entities_df.columns = ["text", "label"]
entities_df = entities_df.drop_duplicates()
entities_df = entities_df.sort_values(by='text')
print(entities_df)
entities_df.to_csv('data/am_one_entities.csv', index=False)

                             text   label
42347               (121).topical  ENTITY
35235              (15–30%),herbs  ENTITY
20575                (21).medical  ENTITY
21005                    (24).228  ENTITY
1340                  (26).toners  ENTITY
...                           ...     ...
2584                  ﬂuorescence  ENTITY
39574  ﬂuorescent dna binding dye  ENTITY
26516                  ﬂuorescing  ENTITY
14029            ﬂushing response  ENTITY
28588                    ﬂutamide  ENTITY

[14672 rows x 2 columns]


In [114]:
# tokens
doc = nlp(all_pages_text.lower())
tokens = []
for token in doc:
    if token.pos_ not in [
        'AUX', 'PUNCT', 'ADV', 'SPACE', 'DET', 'PART', 'ADP', 'CCONJ', 'NUM']:
        tokens.append([token.text, token.pos_, token.tag_])

len(tokens)

68741

In [115]:
tokens_df = pd.DataFrame(tokens)

tokens_df.columns = ["text", "pos", "tag"]
tokens_df = tokens_df.loc[tokens_df['pos'] == 'NOUN']
# del tokens_df["pos"]
# del tokens_df["tag"]
tokens_df = tokens_df.drop_duplicates()
tokens_df = tokens_df.sort_values(by='text')
print(tokens_df.shape)
tokens_df.to_csv('data/am_one_tokens.csv', index=False)

(6840, 3)


In [126]:
tokens_df = pd.DataFrame(tokens)

tokens_df.columns = ["text", "pos", "tag"]

tokens_df["len"] = tokens_df["text"].apply(lambda x: len(x))
tokens_df["is_alpha"] = tokens_df["text"].apply(lambda x: x.isalpha())
tokens_df = tokens_df.loc[tokens_df['pos'] == 'NOUN']
tokens_df = tokens_df.loc[tokens_df['len'] > 2]
tokens_df = tokens_df.loc[tokens_df['len'] < 20]
tokens_df = tokens_df.loc[tokens_df['is_alpha']]
# print(tokens_df.head())


del tokens_df["pos"]
del tokens_df["tag"]
foo = tokens_df["text"].value_counts()
foo = foo.reset_index()
foo.columns = ['text', 'count']
foo.head()
foo.to_csv('data/am_one_token_frequency.csv', index=False)
"""
"""
# tokens_df.is_alpha.value_counts()

'\n'

In [129]:
# nlp = spacy.load("en_core_web_sm")
# doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
chuncks = []
for chunk in doc.noun_chunks:
    # print(chunk.text)
    chuncks.append(chunk.text)
    
# len(chuncks)
len(list(set(chuncks)))

8975

In [136]:
chuncks_df = pd.DataFrame(chuncks)
chuncks_df.columns = ["text"]

chuncks_df["len"] = chuncks_df["text"].apply(lambda x: len(x))
# chuncks_df["is_alpha"] = chuncks_df["text"].apply(lambda x: x.isalpha())
chuncks_df = chuncks_df.loc[chuncks_df['len'] > 2]
chuncks_df = chuncks_df.loc[chuncks_df['len'] < 30]
# chuncks_df = chuncks_df.loc[chuncks_df['is_alpha']]
chuncks_df = chuncks_df.drop_duplicates()

print(chuncks_df.shape)
chuncks_df.to_csv("data/am_one_chuncks.csv", index=False)

(7858, 2)


In [7]:
# creating a pdf reader object 
pdfReader = PyPDF4.PdfFileReader(pdfFileObj) 
  
# printing number of pages 
print(pdfReader.numPages) 

451


In [8]:
# creating a page object 
pageObj = pdfReader.getPage(26) 
  
# extracting text from page 
print(pageObj.extractText()) 

Thenextsectionofthebookdiscussesformulationdevelopmentandapplicationin
thebasicskincareareas:cleansers,toners,moisturizers,andantiperspirants.These

chaptersareallwrittenbyresearchanddevelopmentscientistsinindustrywithan

understandingofhowtheseproductsfunction.Thechapterspresentthebasicanatomy

andphysiologyoftheskinimpactedbytheproduct,ingredients,keyconsiderations,and

methodsforproductevaluationandtesting.Thedermatologicperspectiveontheuseand

selectionoftheseskincareproductsisalsopresented.
Lastly,thebookpresentsanup-to-datelookatmanyoftheactiveproductsthatform
thecosmeceuticalarenatoinclude:sunscreens,skinlighteningagents,exfoliants,and
anti-agingskincareproducts.Thedermatologicperspectiveoneachoftheseareasfollows

withadiscussionofsunscreensinrelationtoskincancerprevention,theimpactof

cosmeceuticalsontheskin,medicaltherapiesforskinlightening,andacnetreatment

methodologies.Thisapproachallowsthedermatologisttobetterunderstandhowthese

productsareconstructed,butalsohelpstheindust

# New heading